In [33]:
import numpy as np
import copy

def read_text(file):
    with open(file,'r') as f:
        for line in f:
            print(line)

def get_number_of_vertices(mesh):
    string_to_match = 'Vertices'
    for index, line in enumerate(mesh):
        line = line.rstrip()  # Need to remove the \n at the end of the line
        if line == string_to_match:
            print("Number of vertices: ", mesh[index+1])
            break
    
    return [index,int(mesh[index+1])]

def get_number_of_triangles(mesh):
    string_to_match = 'Triangles'
    for index, line in enumerate(mesh):
        line = line.rstrip() 
        if line == string_to_match:
            print("Number of triangles: ", mesh[index+1])
            break
    
    return [index,int(mesh[index+1])]

# Symmetry operation

# Symmetry around the xy-plane means that each point x = (x1, x2, x3), will be mapped onto (x1,x2,-x3)
def symmetry_xy_plane(list_of_triangles,list_of_vertices, number_of_vertices):
    new_vertices = copy.deepcopy(list_of_vertices)
    new_vertices[:,2]*=-1

    new_triangles = copy.deepcopy(list_of_triangles)
    new_triangles[:,0]+=number_of_vertices
    new_triangles[:,1]+=number_of_vertices
    new_triangles[:,2]+=number_of_vertices
    return[new_vertices, new_triangles]

# Symmetry around the xz-plane means that each point x = (x1, x2, x3), will be mapped onto (x1,-x2,x3)
def symmetry_xz_plane(list_of_triangles,list_of_vertices, number_of_vertices):
    new_vertices = copy.deepcopy(list_of_vertices)
    new_vertices[:,1]*=-1

    new_triangles = copy.deepcopy(list_of_triangles)
    new_triangles[:,0]+=number_of_vertices
    new_triangles[:,1]+=number_of_vertices
    new_triangles[:,2]+=number_of_vertices
    return[new_vertices, new_triangles]

# Symmetry around the yz-plane means that each point x = (x1, x2, x3), will be mapped onto (-x1,x2,x3)
def symmetry_yz_plane(list_of_triangles,list_of_vertices,number_of_vertices):
    new_vertices = copy.deepcopy(list_of_vertices)
    new_vertices[:,0]*=-1

    new_triangles = copy.deepcopy(list_of_triangles)
    new_triangles[:,0]+=number_of_vertices
    new_triangles[:,1]+=number_of_vertices
    new_triangles[:,2]+=number_of_vertices
    return[new_vertices, new_triangles]
    

    

# Open mesh to "symmetrize"
mymesh = open('test.mesh','r')
mesh_text = mymesh.readlines()

# Getting t
[index_vertices,number_of_vertices] = get_number_of_vertices(mesh_text)
[index_triangles,number_of_triangles] = get_number_of_triangles(mesh_text)
print(number_of_vertices,index_vertices,'vertices and',number_of_triangles,index_triangles, 'triangles')

# Need to initialise these lists so that np.vstack works properly

list_of_triangles = np.empty(shape = [0,4])
new_vertices = np.empty(shape = [0,4])
new_triangles = np.empty(shape = [0,4])



# Obtaining the list of vertices:
# From the index and num
def get_list_of_vertices(number_of_vertices, index_vertices, mesh_text):
    list_of_vertices = np.empty(shape = [0,4])
    for i in range(number_of_vertices):
        x = np.array(mesh_text[index_vertices+i+2].rstrip().split())
        y = x.astype(float)
        list_of_vertices = np.vstack((list_of_vertices, y))

# Obtaining the list of triangles
for i in range(number_of_triangles):
    x_t = np.array(mesh_text[index_triangles+i+2].rstrip().split())
    y_t = x_t.astype(int)
    list_of_triangles = np.vstack((list_of_triangles, y_t))



[new_vertices, new_triangles] = symmetry_yz_plane(list_of_triangles,list_of_vertices,number_of_vertices)
# print(new_triangles)
# print(list_of_triangles)


Number of vertices:  3

Number of triangles:  1

3 3 vertices and 1 10 triangles


In [34]:
out_file = 'out.mesh'

def write_symetrised_mesh(out_file):    
    f = open(out_file, 'w')
    f.write(mesh_text[0])
    f.write(mesh_text[1])
    f.write(mesh_text[2])
    f.write('Vertices\n')
    f.write(str(2*number_of_vertices)+'\n')
    for line in list_of_vertices:
        y=line.astype(str)
        f.write(y[0]+ ' '+ y[1] + ' ' + y[2]+ ' ' + y[3]+ '\n')
    for line in new_vertices:
        y=line.astype(str)
        f.write(y[0]+ ' '+ y[1] + ' ' + y[2]+ ' ' + y[3]+ '\n')

    f.write('\n')
    f.write('Triangles'+ '\n')
    f.write(str(2*number_of_triangles)+ '\n')

    for line in list_of_triangles:
        y = [int(i) for i in line]
        f.write(str(y[0])+ ' '+ str(y[1]) + ' ' + str(y[2])+ ' ' + str(y[3])+ '\n')

    for line in new_triangles:
        y = [int(i) for i in line]
        f.write(str(y[0])+ ' '+ str(y[1]) + ' ' + str(y[2])+ ' ' + str(y[3])+ '\n')

    f.write("End"+'\n')
    f.close()

    # Reading what I just created

    mymesh = open('test.mesh','r')
    # read_text('out.mesh')


write_symetrised_mesh(out_file)

In [1]:
import meshio
msh = meshio.read("mesh_out.mesh")
meshio.write("mesh.xdmf",meshio.Mesh(points=msh.points, cells={"triangle":msh.cells_dict["triangle"]}))